In [1]:
import tensorflow as tf
import os
import numpy as np
tf.compat.v1.disable_eager_execution()

def get_tfrecords_example(feature, label):
    tfrecords_features = {
        'feature': tf.train.Feature(int64_list=tf.train.Int64List(value=feature)),
        'label': tf.train.Feature(float_list=tf.train.FloatList(value=label))
    }
    
    return tf.train.Example(
        features=tf.train.Features(feature=tfrecords_features))

In [2]:
def to_tfrecords(file, save_dir):
    print("Process To tfrecord File: %s ..." % file)
    num = 0
    writer = tf.io.TFRecordWriter(save_dir + "/" + "part-0000" + str(num) + ".tfrecords")
    lines = open(file)
    for i, line in enumerate(lines):
        tmp = line.strip().split(",")
        feature = [int(tmp[0]), int(tmp[1])]
        label = [float(1) if float(tmp[2]) >= 3 else float(0)]
        example = get_tfrecords_example(feature, label)
        writer.write(example.SerializeToString())
        if (i+1) % 200000 == 0:
            writer.close()
            num += 1
            writer = tf.io.TFRecordWriter(save_dir + "/" + "part-0000" + str(num) + ".tfrecords")
    print("Process To tfrecord File: %s End" % file)
    writer.close()

In [3]:
# metaclass=Singleton
class PS:
    def __init__(self, embedding_dim):
        np.random.seed(2020)
        self.params_server = dict()
        self.dim = embedding_dim
        print("ps inited...")
        
    def pull(self, keys):  # 从参数服务器拉去特征所对应的参数
        values = []
        # 这里传进来的数据是[batch, feature_len]->一个样本的数据，样本的特征长度
        for k in keys:
            tmp = []
            for arr in k:
                value = self.params_server.get(arr, None)
                if value is None:
                    value = np.random.rand(self.dim)
                    self.params_server[arr] = value
                tmp.append(value)
            values.append(tmp)
        
        return np.asarray(values, dtype='float32')
    
    def push(self, keys, values):
        for i in range(len(keys)):
            for j in range(len(keys[i])):  # [batch, feature_len]
                self.params_server[keys[i][j]] = values[i][j]
    
    
    def delete(self, keys):
        for k in keys:
            self.params_server.pop(k)
            
    def save(self, path):
        print("总共包含keys： ", len(self.params_server))
        writer = open(path, "w")
        for k, v in self.params_server.items():
            writer.write(str(k) + "\t" + ",".join(["%.8f" % _ for _ in v]) + "\n")
        writer.close()

In [4]:
class InputFn:
    
    def __init__(self, local_ps):
        self.feature_len = 2
        self.label_len = 1
        self.n_parse_threads = 4
        self.shuffle_buffer_size = 1024
        self.prefetch_buffer_size = 1
        self.batch = 8
        self.local_ps = local_ps
        
    def input_fn(self, data_dir, is_test=False):
        def _parse_example(example):
            features = {
                "feature": tf.io.FixedLenFeature(self.feature_len, tf.int64),
                "label": tf.io.FixedLenFeature(self.label_len, tf.float32),
            }
            return tf.io.parse_single_example(example, features)
        
        def _get_embedding(parsed):
            keys = parsed["feature"]
            keys_array = tf.compat.v1.py_func(self.local_ps.pull, [keys], tf.float32)
            result = {
                "feature": parsed["feature"],
                "label": parsed["label"]
            }
            return tf.io.parse_single_example(example, features)
        
        def _get_embedding(parsed):
            keys = parsed["feature"]
            keys_array = tf.compat.v1.py_func(self.local_ps.pull, [keys], tf.float32)
            result = {
                "feature": parsed["feature"],
                "label": parsed["label"],
                "feature_embedding": keys_array,
            }
            return result
        
        file_list = os.listdir(data_dir)
        files = []
        for i in range(len(file_list)):
            files.append(os.path.join(data_dir, file_list[i]))
        
        dataset = tf.compat.v1.data.Dataset.list_files(files)
        # 数据复制多少份
        if is_test:
            dataset = dataset.repeat(1)
        else:
            dataset = dataset.repeat()
        # 读取tfrecord数据
        dataset = dataset.interleave(
            lambda _: tf.compat.v1.data.TFRecordDataset(_),
            cycle_length=1
        )
        # 对tfrecord的数据进行解析
        dataset = dataset.map(
            _parse_example,
            num_parallel_calls=self.n_parse_threads)
        
        # batch data
        dataset = dataset.batch(
            self.batch, drop_remainder=True)
        
        dataset = dataset.map(
            _get_embedding,
            num_parallel_calls=self.n_parse_threads)
        
        # 对数据进行打乱
        if not is_test:
            dataset.shuffle(self.shuffle_buffer_size)
            
        # 数据预加载
        dataset = dataset.prefetch(
            buffer_size=self.prefetch_buffer_size)
        
        # 迭代器
        iterator = tf.compat.v1.data.make_initializable_iterator(dataset)
        return iterator, iterator.get_next()

In [5]:
import tensorflow as tf

batch = 32
embedding_dim = 8
learning_rate = 0.001

def mf_fn(inputs, is_test):
    # 取特征和y值，feature为：user_id和movie_id
    embed_layer = inputs["feature_embedding"]  # [batch, 2, embedding_dim]
    embed_layer = tf.reshape(embed_layer, shape=[-1, 2, embedding_dim])
    label = inputs["label"]  # [batch, 1]
    # 切分数据，获得user_id的embedding和movie_id的embedding
    embed_layer = tf.split(embed_layer, num_or_size_splits=2, axis=1)
    user_id_embedding = tf.reshape(embed_layer[0], shape=[-1, embedding_dim])
    movie_id_embedding = tf.reshape(embed_layer[1], shape=[-1, embedding_dim])
    # 根据公式进行乘积并求和
    out_ = tf.reduce_mean(
        user_id_embedding * movie_id_embedding, axis=1)  # [batch]
    # 设定预估部分
    out_tmp = tf.sigmoid(out_)   # batch
    if is_test:
        tf.compat.v1.add_to_collections("input_tensor", embed_layer)
        tf.compat.v1.add_to_collections("output_tensor", out_tmp)
        
    # 损失函数loss
    label_ = tf.reshape(label, [-1]) # [batch]
    loss_ = tf.reduce_sum(tf.square(label_ - out_)) # 1
    
    out_dic = {
        "loss": loss_,
        "ground_truth": label_,
        "prediction": out_
    }
    
    return out_dic

# 定义整个图结构，并给出梯度更新方式
def setup_graph(inputs, is_test=False):
    result = {}
    with tf.compat.v1.variable_scope("net_graph", reuse=is_test):
        # 初始模型图
        net_out_dic = mf_fn(inputs, is_test)
        
        loss = net_out_dic["loss"]
        result["out"] = net_out_dic
        
        if is_test:
            return result
        
        # SGD
        emb_grad = tf.gradients(
            loss, [inputs["feature_embedding"]], name="feature_embedding")[0]
        
        result["feature_new_embedding"] = \
            inputs["feature_embedding"] - learning_rate * emb_grad
        
        result["feature_embedding"] = inputs["feature_embedding"]
        result["feature"] = inputs["feature"]
        
        return result

In [6]:
import numpy as np
from sklearn.metrics import roc_auc_score

class AUCUtil(object):
    """Summary
    Args:
        ground_truth(list):
        loss(list):
        prediction(list):
    """
    
    def __init__(self):
        self.reset()
        
    def add(self, loss, g=np.array([]), p=np.array([])):
        self.loss.append(loss)
        self.ground_truth += g.flatten().tolist()
        self.prediction += p.flatten().tolist()
        
    def calc(self):
        return {
            "loss_num": len(self.loss),
            "loss": np.array(self.loss).mean(),
            "auc_num": len(self.ground_truth),
            "auc": roc_auc_score(self.ground_truth, self.prediction) if len(self.ground_truth) > 0 else 0,
            "pcoc": sum(self.prediction) / sum(self.ground_truth)
        }
    
    def calc_str(self):
        res = self.calc()
        return "loss: %f(%d), auc: %f(%d), pcoc: %f" % (
            res["loss"], res["loss_num"],
            res["auc"], res["auc_num"],
            res["pcoc"]
        )
    
    def reset(self):
        self.loss = []
        self.ground_truth = []
        self.prediction = []

In [7]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

batch = 32
embedding_dim = 8
local_ps = PS(embedding_dim)
n_parse_threads = 4
shuffle_buffer_size = 1024
prefetch_buffer_size = 16
max_steps = 100000
test_show_step = 1000
# 数据输入
inputs = InputFn(local_ps)

last_test_auc = 0.
# 训练
train_metric = AUCUtil()
test_metric = AUCUtil()

train_file = './save_data/train/'
test_file = './save_data/val/'

saved_embedding = ''

train_itor, train_inputs = inputs.input_fn(train_file, is_test=False)
train_dic = setup_graph(train_inputs, is_test=False)

test_itor, test_inputs = inputs.input_fn(test_file, is_test=True)
test_dic = setup_graph(test_inputs, is_test=True)

train_log_iter = 1000
last_test_auc = 0.5

def train():
    _step = 0
    print("#" * 80)
    # 建立sess,进行训练
    with tf.compat.v1.Session() as sess:
        # init global & local variables
        sess.run([tf.compat.v1.global_variables_initializer(),
                  tf.compat.v1.local_variables_initializer()])
        # 开始训练
        sess.run(train_itor.initializer)
        while _step < max_steps:
            feature_old_embedding, feature_new_embedding, keys, out = sess.run(
                [train_dic["feature_embedding"],
                 train_dic["feature_new_embedding"],
                 train_dic["feature"],
                 train_dic["out"]]
            )
            
            train_metric.add(
                out["loss"],
                out["ground_truth"],
                out["prediction"])
            
            local_ps.push(keys, feature_new_embedding)
            _step += 1
            
            # 每次训练多少个batch的训练数，就打印一次训练的这些batch的auc的信息
            if _step % train_log_iter == 0:
                print("Train at step %d: %s", _step, train_metric.calc_str())
                train_metric.reset()
            if _step % test_show_step == 0:
                valid_step(sess, test_itor, test_dic)

def valid_step(sess, test_itor, test_dic):
    test_metric.reset()
    sess.run(test_itor.initializer)
    global last_test_auc
    while True:
        try:
            out = sess.run(test_dic["out"])
            
            test_metric.add(
                out["loss"],
                out["ground_truth"],
                out["prediction"])
            
        except tf.errors.OutOfRangeError:
            print("Test at step: %s", test_metric.calc_str())
            if test_metric.calc()["auc"] > last_test_auc:
                last_test_auc = test_metric.calc()["auc"]
                local_ps.save(saved_embedding)
                
            break

ps inited...
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


In [ ]:
if __name__ == "__main__":
    train()

################################################################################
Train at step %d: %s 1000 loss: 3.842416(1000), auc: 0.502888(8000), pcoc: 0.303597
Test at step: %s loss: 3.841778(1200823), auc: 0.498068(9606584), pcoc: 0.304140
Train at step %d: %s 2000 loss: 3.811954(1000), auc: 0.492183(8000), pcoc: 0.307573


In [ ]:
import numpy as np

def bkdr2hash64(str01):
    mask60 = 0x0fffffffffffffff
    seed = 131
    hash = 0
    for s in str01:
        hash = hash * seed + ord(s)
    return hash & mask60

# 读取文件
def read_embedding_file(file):
    dic = dict()
    with open(file) as f:
        for line in f:
            tmp = line.split("\t")
            embedding = [float(_) for _ in tmp[1].split(",")]
            dic[tmp[0]] = embedding
    return dic

def get_hash2id(file):
    movie_dict = {}
    user_dict = {}
    with open(file) as f:
        for line in f:
            tmp = line.split(",")
            movie_dict[str(bkdr2hash64("UserID=" + tmp[1]))] = tmp[1]
            user_dict[str(bkdr2hash64("MovieID=" + tmp[0]))] = tmp[0]
    return user_dict, movie_dict

def split_user_movie(embedding_file, train_file):
    user_dict, movie_dict = get_hash2id(train_file)
    embedding_dict = read_embedding_file(embedding_file)
    
    movie_embedding = {}
    user_embedding = {}
    
    for k, v in embedding_dict.items():
        m_id = movie_dict.get(k, None)
        if m_id is not None:
            movie_embedding[m_id] = v
        u_id = user_dict.get(k, None)
        if u_id is not None:
            user_embedding[u_id] = v
            
    return movie_embedding, user_embedding

# 用于i2i模式
def col_sim(movie_sim_movie_file, movie_embedding):
    wfile = open(movie_sim_movie_file, "w")
    for m, vecl in movie_embedding.items():
        sim_movie_tmp = {}
        for n, vec2 in movie_embedding.items():
            if m == n:
                continue
            sim_movie_tmp[n] = np.dot(np.asarray(vec2), np.asarray(vec1))
            
        sim_movie = sorted(sim_movie_tmp.items(), key=lambda _:_[1], reverse=True)
        sim_movie = [str(_[0]) for _ in sim_movie][: 200]
        
        wfile.write(m + "\t" + ",".join(sim_movie) + "\n")
        
# 用u2i模式和排序
def write_user_movie_embeding(movie_embedding_file, user_embedding_file, movie_embedding, user_embedding):
    wfile01 = open(movie_embedding_file, "w")
    for k, v in movie_embedding.items():
        wfile01.write(k + "\t" + ",".join([str(_) for _ in v]) + "\n")
    
    wfile01.close()
    wfile02 = open(user_embedding_file, "w")
    for k, v in user_embedding.items():
        wfile02.write(k + "\t" + ",".join([str(_) for _ in v]) + "\n")
    wfile02.close()

In [ ]:
if __name__ == "__main__":
    embedding_file = './save_data/saved_embedding'
    train_file = './save_data/train_set'
    movie_embedding, user_embedding = split_user_movie(embedding_file, train_file)
    
    # 用于u2i模式和排序
    movie_embedding_file = './save_data/movie_embedding_file'
    user_embedding_file = './save_data/user_embedding_file'
    write_user_movie_embeding(movie_embedding_file, user_embedding_file, movie_embedding, user_embedding)
    
    # 用于i2i模式
    movie_embedding_file = './save_data/movie_sim_movie_file'
    col_sim(movie_sim_movie_file, movie_embedding)